In [ ]:
from nltk.corpus import wordnet as wn

forms = set() #We'll store the derivational forms in a set to eliminate duplicates
for happy_lemma in wn.lemmas(""): #for each "happy" lemma in WordNet
    forms.add(happy_lemma.name()) #add the lemma itself
    for related_lemma in happy_lemma.derivationally_related_forms(): #for each related lemma
        forms.add(related_lemma.name()
        
print(forms))

In [ ]:
from neo4j import GraphDatabase

class nGraph():
    def __init__(self, uri, user, password):
        self._driver = GraphDatabase.driver(uri, auth=(user, password))
    def close(self):
        self._driver.close()
    

    def __run_statement(self, query):
        with self._driver.session() as session:
            return session.write_transaction(self.run, query)

    def run_raw_query(self, query):
        return self.__run_statement(query)

    def get_label(self, entity):
        query = "MATCH (n:ObjectEntity{objectEntity:'"+entity+"'}) RETURN labels(n) as lb"
        r = self.__run_statement(query)[1].records()
        for n in r:
            return n['lb']

    def check_node_exist(self, node_entity):
        cypher_s = "MATCH (n:ObjectEntity{objectEntity:'"+node_entity+"'}) RETURN count(n) <> 0 as res"
        r = self.__run_statement(cypher_s)[1].records()
        for n in r:
            return n['res']
    
    def get_direct_meaning_node(self, entity, r_type):
        query = "MATCH(n:ObjectEntity{objectEntity:'"+entity+"'})-[r:"+r_type+"]-(v:ROOTNODE) return v.objectEntity as root"
        r = self.__run_statement(query)[1].records()
        for n in r:
            return n['root']

    

    def check_no_direct_vn_meaning(self, node_entity):
        cypher_s = "match(n:ObjectEntity{objectEntity:'"+node_entity+"'})-[r:TRANS_EN_VI]-() return count(r) = 0 as res"
        r = self.__run_statement(cypher_s)[1].records()
        for n in r:
            return n['res']

    def get_meaning_of_word(self, w, pos):
        query = "match (n:ObjectEntity{objectEntity:'"+w+"'})-[r:TRANS_EN_VI|:EXPLAINAION_VI_VI]->(v"+pos+")  return n,r,v"
        return self.__run_statement(query)
    def get_meaning_itself(self, w):
        query = "match (n:ObjectEntity{objectEntity:'"+w+"'})-[:LANG_POLY_MEANING]->(v) where exists(v.definition)  return labels(v) as lbs, v.definition as def ,ID(v) as id"
        return self.__run_statement(query)
    def get_en_synonynm(self, _id):
        query = 'match (n)-[:LANG_RELATED_SYNONYM]->(v) where id(n)='+str(_id) + " return v.objectEntity as s"
        return self.__run_statement(query)

    def __getitem__(self, key):
        cypher_s = "MATCH (n {name: '"+key+"'}) return n" 
        return self.__run_statement(cypher_s)
    
    @staticmethod
    def run(tx, message):
        result = tx.run(message)
        avail = result.summary().result_available_after
        cons = result.summary().result_consumed_after
        total_time = avail + cons
        return str(total_time)+" ms", result

In [ ]:
from nltk.stem import PorterStemmer
e_words= ["abolitionizing"]
ps =PorterStemmer()
for w in e_words:
    rootWord=ps.stem(w)
    print(rootWord)

In [85]:
from nltk import word_tokenize, pos_tag
text = word_tokenize("browner")
pos_tag(text)

[('browner', 'NN')]

In [ ]:

g = nGraph("bolt://localhost:7687", "ajax997", "lumia1020")



In [ ]:
results = g.run_raw_query("match (n:ROOTNODE)-[:LANG_GRAMMAR_GERUND]->(v:GERUND) where not (n)-[:LANG_POS_MEANING]-(v) return v.objectEntity as r, labels(v) as lb")[1].records()
for e in results:
    if pos_tag([e['r']])[0][1] == "VBG" and "'" not in e['r']:
        g.run_raw_query("match (n:GERUND{objectEntity:'"+e['r']+"'})-[:LANG_GRAMMAR_GERUND]-(v:ROOTNODE) where not (n)-[:LANG_POS_MEANING]-(v) set n:VBG create (n)-[:LANG_POS_MEANING]->(v)")

        

In [83]:
from pattern.en import pluralize, singularize

results = g.run_raw_query("match (n:ROOTNODE:ObjectEntity)-[:LANG_POLY_MEANING]->(v:NOUN) where not (n)<-[:LANG_POS_MEANING]-(:NNS) and apoc.text.indexOf(n.objectEntity,' ') = -1 and not n:NNS return n.objectEntity as r")[1].records()

for x in results:
    ori = x['r']
    pr = pluralize(ori).replace("'", "\'")
    if pr[len(pr)-2:] != 'ss':
        
            
        g.run_raw_query("merge (n:NNS{objectEntity:\""+pr+"\", language:'english'}) merge (v:ROOTNODE:ObjectEntity{objectEntity:\""+ori+"\"}) merge (n)-[:LANG_POS_MEANING]->(v)")

            

In [ ]:
from pattern.en import pluralize, singularize
print( pluralize('tenderfoot')) #children


In [97]:
from pattern.en import comparative, superlative
superlative('damp')

'dampest'

In [115]:
results = g.run_raw_query("match (n:ObjectEntity)-[:LANG_POLY_MEANING]->(:ADJECTIVE) where right(n.objectEntity, 2) <> 'ed' and right(n.objectEntity, 3) <> 'ing' return DISTINCT n.objectEntity as r")[1].records()


In [116]:
c = 0
for r in results:
    su = superlative(r['r'])
    cp = comparative(r['r'])
    if not su.startswith("most"):
        c+=1
        g.run_raw_query("merge (a:ObjectEntity:ROOTNODE{objectEntity:\""+r['r']+"\"}) merge (b:ObjectEntity:ROOTNODE{objectEntity:\""+cp+"\"}) merge (c:ObjectEntity:ROOTNODE{objectEntity:\""+su+"\"}) set b:JJR, c:JJS merge (c)-[:LANG_POS_MEANING]->(a) merge(b)-[:LANG_POS_MEANING]->(a)")
print(c)

2338


In [123]:
from autocorrect import Speller
spell = Speller(lang='en')
spell("sentime")

'septime'